In [4]:
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

In [11]:
# Load the CSV Data
df= pd.read_csv('../data/handlebar/Accelerometer_data_merged_ESP1.csv')  # Load the CSV data using pandas
print("Index values:", df.index.values)


Index values: [     0      1      2 ... 549232 549233 549234]


In [16]:
# Filter the DataFrame to include only the necessary columns
df_filtered = df[['NTP','Acc-Z']]
df_filtered.head()

,NTP,Acc-Z
0,2024-07-24 10:26:01.863,8.762772
1,2024-07-24 10:26:01.867,8.753204
2,2024-07-24 10:26:01.872,8.762772
3,2024-07-24 10:26:01.877,8.781937
4,2024-07-24 10:26:01.884,8.753204


In [ ]:
def handle_missing_values_length(df_filtered):
    # Handle missing values length
    missing_info = {}  # List to store (start_index, current_count)
    current_count = 0
    start_index = None  # To store the start time of missing values
    for index, row in df_filtered.iterrows():
        if np.isnan(row['Acc-Z']):
            if start_index is None:
                start_index = index
                current_count += 1
            if start_index is not None:
                current_count += 1
        # If the value is not NaN, continue sequence broke
        else:
            if start_index is not None:
                missing_info[start_index] = current_count
                start_index = None
                current_count = 0

    # Handle case where the last segment has missing values
    if current_count > 0:
        missing_info[start_index] = current_count

    for key, value in missing_info.items():
        #print(key, value)
        if value > 100:
            print("Start Index:", key, "Count:", value)  
    print("No missing value longer than half a second")
    
    return missing_info


In [ ]:
missing_info = handle_missing_values_length(df_filtered)


No missing value longer than half a second
